In [1]:
import os
import random
import requests
import pandas as pd
import subprocess
from bs4 import BeautifulSoup

In [2]:
ACCESS_TOKEN = 'access token'

payload_mark = {
    '_qf__login_form': '',
    'user_login': 'email',
    'user_password': 'password'
}


def make_vk_request(method, params={}):
    url = f'https://api.vk.com/method/{method}'
    params['access_token'] = ACCESS_TOKEN
    params['v'] = '5.101'
    return requests.get(url, params=params)


def send_message(msg=None, attachment=None):
    method = 'messages.send'
    params = {
        'random_id': random.getrandbits(63) * random.choice([-1, 1]),
        'domain': 'yngmrk',
    }
    if msg: params['message'] = msg
    if attachment: params['attachment'] = attachment
    return make_vk_request(method, params)


def get_upload_url():
    method = 'photos.getMessagesUploadServer'
    return make_vk_request(method).json()['response']['upload_url']

In [3]:
session = requests.session()
session.post('https://lms.hse.ru/', data=payload_mark, timeout=5)
response_gradebook = session.get('https://lms.hse.ru/index.php?page=gradebook')
soup = BeautifulSoup(response_gradebook.text)

In [16]:
tbl = []
module_2 = soup.find('td', string='4 модуль').parent
row = module_2.next_sibling
while True:
    if 'class' in row.attrs:
        break
    
    l = list(i.text for i in row.children)
    name = l[0]
    score_10 = l[6] if l[6] else '-'
    score_5 = l[7]
    
    tbl.append([name, score_10, score_5])

    row = row.next_sibling

headers = ['Предмет', 'Оценка по 10 б.', 'Оценка по 5 б.']
df = pd.DataFrame(tbl, columns=headers)
gradebook_html = df.to_html(index=False)
with open('gradebook.html', "w", encoding="utf-16") as f:
    f.write(gradebook_html)

path_to_html = os.getcwd() + '\gradebook.html'
proc = subprocess.run(f'd:\\programs\wkhtmltox\wkhtmltoimage -f png --width 0 {path_to_html} gradebook.png',
                      shell=True,
                      capture_output=True)
print(proc.stderr.decode('866'))

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               



In [17]:
upload_url = get_upload_url()

files = {
    'photo': open('gradebook.png', 'rb')
}
upload_request = requests.post(upload_url, files=files)

save_request = make_vk_request('photos.saveMessagesPhoto', upload_request.json())

if save_request.status_code == upload_request.status_code == 200:
    r = save_request.json()['response'][0]
    owner_id = r['owner_id']
    photo_id = r['id']
    send_message(attachment=f"photo{owner_id}_{photo_id}")